In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')
# log_dir='/content/gdrive/MyDrive/DL_Project/runs'
# !mkdir -p {log_dir}
# !mv ./runs/* {log_dir}

Mounted at /content/drive


In [ ]:
!pip install -U git+https://github.com/Adapter-Hub/adapter-transformers.git@v2
!pip install torch==1.7.1


  Cloning https://github.com/Adapter-Hub/adapter-transformers.git (to revision v2) to /tmp/pip-req-build-r1y6d7hf
  Running command git clone -q https://github.com/Adapter-Hub/adapter-transformers.git /tmp/pip-req-build-r1y6d7hf
  Running command git checkout -b v2 --track origin/v2
  Switched to a new branch 'v2'
  Branch 'v2' set up to track remote branch 'v2' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 8.0MB/s 
     |████████████████████████████████| 901kB 33.5MB/s 
  Created wheel for adapter-transformers: filename=adapter_transformers-2.0.0a1-cp37-none-any.whl size=2097547 sha256=de91908ef923c1ddd4ed2ded016c890ece5ac829ab37a0140eb4e192311e8b71
  Stored in directory: /tmp/pip-ephem-wheel-cache-wx7d72j1/wheels/11/c5/35/7017ef1a9923a73e9d8071801894534ab1fa662e38e23b78f1
Successfully built adapter-transformers
     |████████████████████████████

In [ ]:
import json, gc
import urllib.request
import numpy as np
import pandas as pd
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
    RobertaTokenizer
)
from transformers import RobertaForSequenceClassification, RobertaModelWithHeads, RobertaConfig
from transformers import TrainingArguments, Trainer, EvalPrediction

from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from transformers import EarlyStoppingCallback
from transformers.integrations import TensorBoardCallback
from tensorflow import summary
import tensorflow
%load_ext tensorboard
import datetime

Using device: cuda


Dataset and Tokenizers

In [ ]:
DATASET = 'imdb'
tokenizer = RobertaTokenizer.from_pretrained("allenai/reviews_roberta_base")

In [ ]:
print(tokenizer.bos_token_id, tokenizer.bos_token, tokenizer.eos_token_id, tokenizer.eos_token)
MAX_SEQUENCE_LENGTH = 512


def load_and_tokenize(url, label2id={}, count_label = 0, tokenizer=tokenizer):
  # tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
  block_size=512
  dataframe = []
  with urllib.request.urlopen(url) as f:
    for line in f:
      doc = json.loads(line.decode('utf-8'))['text']
      tokenized_text = tokenizer(doc, max_length=MAX_SEQUENCE_LENGTH, truncation=True, padding="max_length")
      label = json.loads(line.decode('utf-8'))['label']
      
      if label not in label2id:
        label2id[label] = count_label
        count_label +=1
      tokenized_text['labels'] = torch.tensor(label2id[label])
      tokenized_text['input_ids'] = torch.tensor(tokenized_text['input_ids'])
      tokenized_text['attention_mask'] = torch.tensor(tokenized_text['attention_mask'])
      dataframe.append(tokenized_text)
  return dataframe, count_label, label2id

train,count_label, label2id = load_and_tokenize("https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/"+DATASET+"/train.jsonl", tokenizer=tokenizer)
dev,count_label,label2id = load_and_tokenize("https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/"+DATASET+"/dev.jsonl",label2id,  count_label, tokenizer=tokenizer)
test,count_label,label2id = load_and_tokenize("https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/"+DATASET+"/test.jsonl",label2id,  count_label, tokenizer=tokenizer)


0 <s> 2 </s>


In [ ]:
# config

In [ ]:
train[0]
print(len(train))

20000


Folder to save results

In [ ]:
RESULTS_DIR = f"""/content/drive/My Drive/DL_Project/results/No Adapter/"""

General packages and functions

In [ ]:
from transformers import TrainingArguments, Trainer, EvalPrediction
def compute_accuracy(p: EvalPrediction):
    labels = p.label_ids
    preds = np.argmax(p.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

from transformers import AdapterType, AdapterConfig, HoulsbyConfig, PfeifferConfig

In [ ]:
def run_model(hyperparams_dict, model_name="roberta-base",task_name="myown",adapter_name=None,adapter_config=None,seed=999):
    global model
    config = RobertaConfig.from_pretrained(
        model_name,
        num_labels=len(label2id),
    )
    model = RobertaModelWithHeads.from_pretrained(
        model_name,
        config=config,
    )

    if torch.cuda.is_available():
      model = model.to("cuda")
    id2label= {v: k for k, v in label2id.items()}
    for param in model.base_model.parameters():
      param.requires_grad = False

    # Add a matching classification head
    model.add_classification_head(
        task_name,
        num_labels=len(label2id),
        id2label=id2label,
        layers=2
      )
      
    '''
    if adapter_name:
      # add a new adapter
      if adapter_config:
        model.add_adapter(
            task_name,
            ##### remove AdapterType argument for v2 #####
            #AdapterType.text_task,
            config=adapter_config
        )
      else:
        model.add_adapter(
            task_name,
            ##### remove AdapterType argument for v2 #####
            #AdapterType.text_task 
        )
      # Enable adapter training
      model.train_adapter([task_name])
      '''

    # train, dev, test = get_datasets(tokenizer)
    training_args = TrainingArguments(
        learning_rate=hyperparams_dict['learning_rate'],
        num_train_epochs=hyperparams_dict['num_train_epochs'],
        per_device_train_batch_size=hyperparams_dict['per_device_train_batch_size'],
        per_device_eval_batch_size=hyperparams_dict['per_device_eval_batch_size'],
        logging_steps=hyperparams_dict['logging_steps'],
        save_steps=hyperparams_dict['save_steps'],
        output_dir='./models/'+task_name,
        overwrite_output_dir=True,
        do_train=True,
        do_eval=True,
        do_predict=True,
        evaluation_strategy='steps', # use evaluation_strategy='epoch' for v2, evaluation_strategy='step' for large dataset
        # The next line is important to ensure the dataset labels are properly passed to the model
        remove_unused_columns=False,
        load_best_model_at_end=True,
        metric_for_best_model="loss",
        greater_is_better=False,
        seed=int(seed)
    )

    # tensor_board = TensorBoardCallback()
    ##### Early Stopping #####
    es = EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.0)
    if adapter_name:
      trainer = Trainer(
          model=model,
          args=training_args,
          train_dataset=train,
          eval_dataset=dev,
          compute_metrics=compute_accuracy,
          callbacks=[es],
          adapter_names=[adapter_name]   
      )
    else:
      trainer = Trainer(
          model=model,
          args=training_args,
          train_dataset=train,
          eval_dataset=dev,
          callbacks=[es],
          compute_metrics=compute_accuracy
      )
    trainer.train()

    ##### Explicitly set active adapter to pass it in model forward pass,             #####
    ##### otherwise the previous setting adapter_names=[adapter_name] not work for v2 #####
    #if adapter_name:
    #  trainer.model.set_active_adapters(adapter_name)

    _, _, metrics = trainer.predict(dev)
    metrics['seed'] = seed
    dev_results.append(pd.DataFrame.from_dict(metrics, orient='index').T)

    metrics['seed'] = seed
    _, _, metrics = trainer.predict(test)
    test_results.append(pd.DataFrame.from_dict(metrics, orient='index').T)
    
    filepath = RESULTS_DIR + """{}_{}_{}""".format(task_name,seed,timestamp.strftime("%Y-%m-%dT%H_%M_%S"))
    filepath = filepath + '_' + str(hyperparams_dict['learning_rate']) \
                + '_' + str(hyperparams_dict['num_train_epochs']) \
                + '_' + str(hyperparams_dict['per_device_train_batch_size'] ) \
                + '_' + str(hyperparams_dict['per_device_eval_batch_size']) \
                + '_' + str(hyperparams_dict['logging_steps']) \
                + '_' + str(hyperparams_dict['save_steps'])
    trainer.save_model(filepath)

### Model: RoBERTa Base
### Task: Reviews
### Finetuning: Standard with classification head
### Adapter: None (None /Custom /Default / Houlsby / Pfeiffer)

In [ ]:
# MODEL_NAME = "allenai/cs_roberta_base"
MODEL_NAME = "allenai/dsp_roberta_base_tapt_imdb_20000"
TASK_NAME = "imdb_no_adapter_TAPT" # cit_intent_base_myownadapter / cit_intent_base_pfieffer
ADAPTER_NAME = None # None pfieffer / cit_intent_base_finetune


ADAPTER_CONFIG = PfeifferConfig() # leave ADAPTER_CONFIG as None to default adapter

# ADAPTER_CONFIG = AdapterConfig.load( # comment out if using default adapter
#     # adapter_args.adapter_config,
#     # non_linearity=adapter_args.adapter_non_linearity,
#     # reduction_factor=adapter_args.adapter_reduction_factor,
#     ADAPTER_NAME, 
#     # non_linearity=adapter_args.adapter_non_linearity,
#     reduction_factor=12
# )

# hyperparameters search
hyperparameters_dict = {'learning_rate':2e-5,'num_train_epochs':3,'per_device_train_batch_size':16,'per_device_eval_batch_size':16,'logging_steps':1000,'save_steps':1000} # hyperparameters for standard finetuning
'''
HYPERPARAMETERS_SEARCH = [{'learning_rate':1e-4,'num_train_epochs':10,'per_device_train_batch_size':16,'per_device_eval_batch_size':16,'logging_steps':1000,'save_steps':1000},
                          {'learning_rate':8e-5,'num_train_epochs':10,'per_device_train_batch_size':16,'per_device_eval_batch_size':16,'logging_steps':1000,'save_steps':1000},
                          {'learning_rate':2e-5,'num_train_epochs':10,'per_device_train_batch_size':16,'per_device_eval_batch_size':16,'logging_steps':1000,'save_steps':1000},
                          {'learning_rate':1e-4,'num_train_epochs':10,'per_device_train_batch_size':32,'per_device_eval_batch_size':32,'logging_steps':1000,'save_steps':1000},
                          {'learning_rate':8e-5,'num_train_epochs':10,'per_device_train_batch_size':32,'per_device_eval_batch_size':32,'logging_steps':1000,'save_steps':1000},
                          {'learning_rate':2e-5,'num_train_epochs':10,'per_device_train_batch_size':32,'per_device_eval_batch_size':32,'logging_steps':1000,'save_steps':1000},
                          ]
'''
HYPERPARAMETERS_SEARCH = [ {'learning_rate':24e-5,'num_train_epochs':3,'per_device_train_batch_size':16,'per_device_eval_batch_size':16,'logging_steps':1000,'save_steps':1000}]


In [ ]:


from datetime import datetime, timedelta
timestamp = datetime.today() + timedelta(hours=8)
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
print(tokenizer.bos_token_id, tokenizer.bos_token, tokenizer.eos_token_id, tokenizer.eos_token)

seeds = [42, 2,1]
dev_results = []
test_results = []
print('seeds:', seeds)

if ADAPTER_NAME:
    # if using adapter, loop by HYPERPARAMETERS_SEARCH defined above
    for hyperparameters_dict in HYPERPARAMETERS_SEARCH:
        gc.collect()
        torch.cuda.empty_cache()
        print(hyperparameters_dict)
        run_model(hyperparameters_dict, model_name=MODEL_NAME,task_name=TASK_NAME,adapter_name=ADAPTER_NAME,adapter_config=ADAPTER_CONFIG)
else: # if not using adapter, assume standard finetuning and loop by seeds
    for seed in seeds:
        print(type(int(seed)))
        gc.collect()
        torch.cuda.empty_cache()
        print(hyperparameters_dict)
        run_model(hyperparameters_dict, model_name=MODEL_NAME,task_name=TASK_NAME,adapter_name=ADAPTER_NAME,adapter_config=ADAPTER_CONFIG, seed=seed)

dev_df = pd.concat(dev_results)
test_df = pd.concat(test_results)


# UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior._warn_prf(average, modifier, msg_start, len(result))
## some labels in y_test don't appear in y_pred. 


0 <s> 2 </s>
seeds: [42, 2, 1]
<class 'int'>
{'learning_rate': 4e-05, 'num_train_epochs': 3, 'per_device_train_batch_size': 16, 'per_device_eval_batch_size': 16, 'logging_steps': 1000, 'save_steps': 1000}


Some weights of the model checkpoint at allenai/dsp_roberta_base_tapt_imdb_20000 were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at allenai/dsp_roberta_base_tapt_imdb_20000 and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-str

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1000,0.592500,0.505109,0.852000,0.851748,0.854406,0.852000,89.435500,55.906000
2000,0.442800,0.399718,0.864600,0.864402,0.866741,0.864600,89.451600,55.896000
3000,0.386900,0.360790,0.875600,0.875593,0.875687,0.875600,89.482500,55.877000


<class 'int'>
{'learning_rate': 4e-05, 'num_train_epochs': 3, 'per_device_train_batch_size': 16, 'per_device_eval_batch_size': 16, 'logging_steps': 1000, 'save_steps': 1000}


Some weights of the model checkpoint at allenai/dsp_roberta_base_tapt_imdb_20000 were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at allenai/dsp_roberta_base_tapt_imdb_20000 and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-str

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1000,0.594600,0.509200,0.843600,0.843035,0.848620,0.843600,89.473400,55.883000
2000,0.444200,0.402518,0.866800,0.866747,0.867388,0.866800,89.484400,55.876000
3000,0.390200,0.364844,0.874600,0.874591,0.874711,0.874600,89.455600,55.894000


<class 'int'>
{'learning_rate': 4e-05, 'num_train_epochs': 3, 'per_device_train_batch_size': 16, 'per_device_eval_batch_size': 16, 'logging_steps': 1000, 'save_steps': 1000}


Some weights of the model checkpoint at allenai/dsp_roberta_base_tapt_imdb_20000 were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at allenai/dsp_roberta_base_tapt_imdb_20000 and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-str

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1000,0.595400,0.508210,0.852800,0.852779,0.853003,0.852800,89.530100,55.847000
2000,0.449500,0.405382,0.863200,0.863030,0.865009,0.863200,89.463200,55.889000
3000,0.392500,0.367134,0.875200,0.875200,0.875201,0.875200,89.474900,55.882000


In [ ]:
display(dev_df)
display(dev_df.describe())

display(test_df)
display(test_df.describe())

,test_loss,test_accuracy,test_f1,test_precision,test_recall,test_runtime,test_samples_per_second,test_mem_cpu_alloc_delta,test_mem_gpu_alloc_delta,test_mem_cpu_peaked_delta,test_mem_gpu_peaked_delta,seed
0,0.360790,0.8756,0.875593,0.875687,0.8756,89.4665,55.887,0.0,0.0,0.0,780468736.0,42.0
0,0.364844,0.8746,0.874591,0.874711,0.8746,89.5332,55.845,-7430144.0,0.0,7430144.0,780463616.0,2.0
0,0.367134,0.8752,0.875200,0.875201,0.8752,89.4713,55.884,0.0,0.0,0.0,780462080.0,1.0


,test_loss,test_accuracy,test_f1,test_precision,test_recall,test_runtime,test_samples_per_second,test_mem_cpu_alloc_delta,test_mem_gpu_alloc_delta,test_mem_cpu_peaked_delta,test_mem_gpu_peaked_delta,seed
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000e+00,3.0,3.000000e+00,3.000000e+00,3.000000
mean,0.364256,0.875133,0.875128,0.875200,0.875133,89.490333,55.872000,-2.476715e+06,0.0,2.476715e+06,7.804648e+08,15.000000
std,0.003213,0.000503,0.000505,0.000488,0.000503,0.037201,0.023431,4.289796e+06,0.0,4.289796e+06,3.485112e+03,23.388031
min,0.360790,0.874600,0.874591,0.874711,0.874600,89.466500,55.845000,-7.430144e+06,0.0,0.000000e+00,7.804621e+08,1.000000
25%,0.362817,0.874900,0.874895,0.874956,0.874900,89.468900,55.864500,-3.715072e+06,0.0,0.000000e+00,7.804628e+08,1.500000
50%,0.364844,0.875200,0.875200,0.875201,0.875200,89.471300,55.884000,0.000000e+00,0.0,0.000000e+00,7.804636e+08,2.000000
75%,0.365989,0.875400,0.875396,0.875444,0.875400,89.502250,55.885500,0.000000e+00,0.0,3.715072e+06,7.804662e+08,22.000000
max,0.367134,0.875600,0.875593,0.875687,0.875600,89.533200,55.887000,0.000000e+00,0.0,7.430144e+06,7.804687e+08,42.000000


,test_loss,test_accuracy,test_f1,test_precision,test_recall,test_runtime,test_samples_per_second,test_mem_cpu_alloc_delta,test_mem_gpu_alloc_delta,test_mem_cpu_peaked_delta,test_mem_gpu_peaked_delta
0,0.358486,0.87312,0.873119,0.873126,0.87312,447.3517,55.884,0.0,0.0,0.0,8.308654e+08
0,0.362691,0.87220,0.872184,0.872383,0.87220,447.4636,55.870,811008.0,0.0,0.0,1.107992e+09
0,0.364992,0.87204,0.872039,0.872053,0.87204,447.3014,55.891,0.0,0.0,0.0,1.133180e+09


,test_loss,test_accuracy,test_f1,test_precision,test_recall,test_runtime,test_samples_per_second,test_mem_cpu_alloc_delta,test_mem_gpu_alloc_delta,test_mem_cpu_peaked_delta,test_mem_gpu_peaked_delta
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.0,3.0,3.000000e+00
mean,0.362057,0.872453,0.872448,0.872521,0.872453,447.372233,55.881667,270336.000000,0.0,0.0,1.024012e+09
std,0.003299,0.000583,0.000586,0.000550,0.000583,0.083027,0.010693,468235.687115,0.0,0.0,1.677437e+08
min,0.358486,0.872040,0.872039,0.872053,0.872040,447.301400,55.870000,0.000000,0.0,0.0,8.308654e+08
25%,0.360589,0.872120,0.872112,0.872218,0.872120,447.326550,55.877000,0.000000,0.0,0.0,9.694285e+08
50%,0.362691,0.872200,0.872184,0.872383,0.872200,447.351700,55.884000,0.000000,0.0,0.0,1.107992e+09
75%,0.363842,0.872660,0.872652,0.872754,0.872660,447.407650,55.887500,405504.000000,0.0,0.0,1.120586e+09
max,0.364992,0.873120,0.873119,0.873126,0.873120,447.463600,55.891000,811008.000000,0.0,0.0,1.133180e+09


Saving results

In [ ]:

filepath = RESULTS_DIR+TASK_NAME

# save adapter + classifier
if ADAPTER_NAME:
  model.save_adapter(filepath, TASK_NAME)
model.save_head(filepath, TASK_NAME)

# dev and test results + hyperparameters used. hyperparameters will be in the same order dev_ and test_ results are appended
dev_df.to_excel(filepath+'/dev_results.xlsx',index=False)
test_df.to_excel(filepath+'/test_results.xlsx',index=False)
if ADAPTER_NAME:
  with open(filepath+'/hyperparameters.py', 'w') as writefile:
      writefile.write("HYPERPARAMETERS_SEARCH = {}".format(HYPERPARAMETERS_SEARCH))
else:
  with open(filepath+'/hyperparameters.py', 'w') as writefile:
      writefile.write("HYPERPARAMETERS_SEARCH = {}".format(hyperparameters_dict))